# NEWS SCRAPING

### GNews Articles

In [13]:
from datetime import datetime, timedelta, date
import pandas as pd
import time
from gnews import GNews
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_days_from_period(period: str) -> int:
    """
    Converts a period string (e.g., '7d', '30d', '365d') to the number of days.

    Args:
    - period (str): The period string.

    Returns:
    - int: The number of days corresponding to the given period.
    """
    period_unit = period[-1]  # 'd' for days
    period_value = int(period[:-1])  # Extract the numeric part

    if period_unit == 'd':
        return period_value
    else:
        raise ValueError("Invalid period format. Use 'd' for days.")

def fetch_news_for_interval(company_name: str, interval_start: str, interval_end: str, max_results: int = 100):
    """
    Fetch news articles for a specific time interval.

    Args:
    - company_name (str): The name of the company to search for.
    - interval_start (str): The start date in 'YYYY-MM-DD' format.
    - interval_end (str): The end date in 'YYYY-MM-DD' format.
    - max_results (int): The maximum number of results for this interval.

    Returns:
    - articles (list): List of dictionaries containing news articles for the interval.
    """
    logger.info(f"Fetching articles for the period: {interval_start} to {interval_end}")
    google_news = GNews(language='en', country='IN', max_results=max_results, start_date=date.fromisoformat(interval_start), end_date=date.fromisoformat(interval_end))
    search_results = google_news.get_news(company_name)
    articles = []
    for result in search_results:
        try:
            link = result["url"]
            date_published = datetime.strptime(result["published date"], '%a, %d %b %Y %H:%M:%S GMT').strftime('%Y-%m-%d')
            title = result["title"]
            if not title:
                raise ValueError("Title empty.")

            article = {
                "Title": title,
                "Link": link,
                "Published_Date": date_published
            }
            articles.append(article)
        except Exception as e:
            logger.error(f"Error : {e}")

    return articles

def news_scraper(company_name: str, period: str, max_results: int = 100):
    """
    Scrapes news articles from Google News for a given company over a split time period.

    Args:
    - company_name (str): The name of the company to search for.
    - period (str): The time period selected from the frontend (e.g., '7d', '30d', '365d', '730d').
    - max_results (int): The maximum number of results per interval (default: 100).

    Returns:
    - df (pd.DataFrame): A DataFrame containing the scraped news articles.
    """

    # Convert the period to the total number of days
    total_days = get_days_from_period(period)

    # Get the current date as the end date (only date, no time)
    end_dt = datetime.now().date()

    # Calculate the start date based on the total days
    start_dt = end_dt - timedelta(days=total_days)

    # Split the date range into 5 equal intervals to get more results
    num_intervals = 5
    interval_days = total_days // num_intervals

    intervals = []
    for i in range(num_intervals):
        interval_start = (start_dt + timedelta(days=i * interval_days)).strftime('%Y-%m-%d')
        interval_end = (start_dt + timedelta(days=(i + 1) * interval_days)).strftime('%Y-%m-%d')
        if i == num_intervals - 1:  # Ensure the last interval goes up to the end date
            interval_end = end_dt.strftime('%Y-%m-%d')

        intervals.append((interval_start, interval_end))

    # Prepare a DataFrame to store all the results
    full_articles_df = pd.DataFrame()

    # Use ThreadPoolExecutor to fetch results in parallel
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_news_for_interval, company_name, start, end, max_results) for start, end in intervals]
        
        for future in as_completed(futures):
            articles = future.result()
            interval_df = pd.DataFrame(articles).reset_index(drop=True)
            full_articles_df = pd.concat([full_articles_df, interval_df], ignore_index=True)

    # Sort the final DataFrame by the published date
    full_articles_df = full_articles_df.sort_values(by="Published_Date").reset_index(drop=True)

    return full_articles_df

# Example usage
df = news_scraper("Gujarat Toolroom", "730d", 100)

11/21/2024 08:18:00 PM - Fetching articles for the period: 2022-11-22 to 2023-04-17
11/21/2024 08:18:00 PM - Fetching articles for the period: 2023-04-17 to 2023-09-10
11/21/2024 08:18:00 PM - Fetching articles for the period: 2023-09-10 to 2024-02-03
11/21/2024 08:18:00 PM - Fetching articles for the period: 2024-02-03 to 2024-06-28
11/21/2024 08:18:00 PM - Fetching articles for the period: 2024-06-28 to 2024-11-21


In [38]:
df.columns

Index(['Title', 'Link', 'Published_Date'], dtype='object')

In [39]:
df

,Title,Link,Published_Date
0,Orient Green okays expansion plan in green ene...,https://news.google.com/rss/articles/CBMiowFBV...,2023-01-04
1,Sheezan Khan Gets Replaced by Abhishek Nigam i...,https://news.google.com/rss/articles/CBMi-AFBV...,2023-01-13
2,Kerala-Based Jeweller Joyalukkas Withdraws IPO...,https://news.google.com/rss/articles/CBMiswFBV...,2023-02-21
3,Not Being Treated With Respect: Air India Pilo...,https://news.google.com/rss/articles/CBMi8AFBV...,2023-04-26
4,Sunny Leone Oozes Oomph in Scorching Hot Green...,https://news.google.com/rss/articles/CBMizgFBV...,2023-06-04
...,...,...,...
132,Q2FY25 results effect: Smallcap stock below ₹5...,https://news.google.com/rss/articles/CBMi9AFBV...,2024-11-12
133,Guj Toolroom Standalone September 2024 Net Sal...,https://news.google.com/rss/articles/CBMi4wFBV...,2024-11-12
134,Penny Stock Under Rs 15: Company Reports Stell...,https://news.google.com/rss/articles/CBMilgJBV...,2024-11-12
135,Gujarat Toolroom Q2 PAT jumps to Rs 27 crore -...,https://news.google.com/rss/articles/CBMijgFBV...,2024-11-13


## RSS link to Actual Link Extraction

#### Playwright Solution (async programming)

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import logging
import time
import pandas as pd

class GoogleNewsLinkResolver:
    def __init__(self, max_concurrent_tasks=10, timeout=5, max_wait=5):
        """
        Initialize the link resolver with configurable parameters.
        
        :param max_concurrent_tasks: Maximum number of concurrent browser tasks
        :param timeout: Page load timeout in seconds
        :param max_wait: Maximum wait time for redirection in seconds
        """
        self.max_concurrent_tasks = max_concurrent_tasks
        self.timeout = timeout * 1000  # Convert to milliseconds for Playwright
        self.max_wait = max_wait
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)

    async def _fetch_single_link(self, link):
        """
        Fetch the original link from a Google News URL.
        
        :param link: Google News link to resolve
        :return: Resolved original link or None if failed
        """
        browser = None
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch(headless=True)
                page = await browser.new_page()
                await page.goto(link, timeout=self.timeout)
                
                redirected_url = page.url
                start_time = asyncio.get_event_loop().time()
                
                while redirected_url.startswith('https://news.google.com'):
                    await asyncio.sleep(0.5)
                    redirected_url = page.url
                    
                    if asyncio.get_event_loop().time() - start_time > self.max_wait:
                        break
                
                await browser.close()
                return redirected_url
        except Exception as e:
            self.logger.error(f"Error resolving link: {e}")
            return None
        finally:
            if browser:
                await browser.close()

    async def resolve_links(self, links):
        """
        Resolve multiple links concurrently.
        
        :param links: List of Google News links
        :return: Dictionary of rss links: resolved links
        """
        semaphore = asyncio.Semaphore(self.max_concurrent_tasks)
        
        async def bounded_fetch(link):
            async with semaphore:
                return await self._fetch_single_link(link)
        
        tasks = [bounded_fetch(link) for link in links]
        results = await asyncio.gather(*tasks)
        
        return dict(zip(links, results))

    def resolve_links_sync(self, links):
        """
        Synchronous wrapper for resolving links.
        
        :param links: List of Google News links
        :return: Dictionary of original links
        """
        return asyncio.run(self.resolve_links(links))

# Example usage
async def main():
    df = pd.read_excel(r"E:\Intern\Minerva\LLM API\TATA Motors.xlsx")
    links = list(df["Link"])
    
    start_time = time.time()
    resolver = GoogleNewsLinkResolver(max_concurrent_tasks=20)
    resolved_links = await resolver.resolve_links(links)
    
    for original_link, resolved_link in resolved_links.items():
        print(f"Resolved: {resolved_link}")
    print("Total time taken: ", time.time()-start_time)
# If running as a script
if __name__ == "__main__":
    asyncio.run(main())

C:\Users\Sahal\AppData\Local\Temp\ipykernel_16536\584484611.py:96: RuntimeWarning: coroutine 'wait' was never awaited
  asyncio.wait(main())
C:\Users\Sahal\AppData\Local\Temp\ipykernel_16536\584484611.py:96: RuntimeWarning: coroutine 'main' was never awaited
  asyncio.wait(main())


## Resolved links to Articles text

In [16]:
from gnews import GNews

df = pd.read_excel(r"E:\Intern\Minerva\Web Scrapping\TATA Motors_LinksResolved.xlsx")
df = df.dropna()

google_news = GNews()

url = df["ResolvedLink"][1]

article = google_news.get_full_article(url)
print(article.text)

The breakup of Volkswagen’s proposed joint venture for making budget cars with Tata Motors in 2017 was seen as the end of the road for German car maker’s India plans by analysts. However, five years later the wheels have turned for the company with new product line-up and sales numbers looking up.

Between January-August of the current calendar this year, Skoda (part of Volkswagen Group) sold more cars than it did in the whole of 2012 which was the best year for the company in India. As for Volkswagen, the current year, is likely to be the best since its debut in the country in 2007.

Both VW and Skoda have together rolled out six models, including four SUVs using the MQB (Modularer Querbaukasten meaning modular transversal toolkit) platform in the last three years. While none of these launches were in the budget category, these introductions happened just when the Indian car market was undergoing yet another pivotal shift.

Also Read Q2 Result 2024: HAL, Hero Motocorp, Muthoot Finance

### Trafilatura attempt

* Trafilatura was found to be faster and more suitable for async calls

In [21]:
import pandas as pd
import asyncio
import aiohttp
import time
from trafilatura import extract

async def fetch_url_async(session, url):
    """Fetch the content of the URL asynchronously using aiohttp."""
    async with session.get(url) as response:
        return await response.text()

async def extract_content(url, session):
    """Fetch and extract content from a URL asynchronously."""
    try:
        # Fetch the page content asynchronously
        page_content = await fetch_url_async(session, url)
        
        # Extract content using trafilatura (this is still blocking)
        tflr_content = extract(page_content)
        
        # Check if content was extracted
        if not tflr_content:
            raise ValueError("Trafilatura failed to extract content")
        
        print(f"Extracted content for {url}:\n{len(tflr_content)}")
        
    except Exception as e:
        print(f"Error URL: {url}, Error: {str(e)}")

# Set up the main logic for the Jupyter Notebook
async def process_urls():
    # Read the DataFrame containing the URLs
    df = pd.read_excel(r"E:\Intern\Minerva\Web Scrapping\TATA Motors_LinksResolved.xlsx")
    df = df.dropna()

    # Set up an aiohttp session
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks for URL processing
        tasks = []
        for url in df["ResolvedLink"]:
            if not url.startswith("https://news.google.com"):
                tasks.append(extract_content(url, session))
        
        # Run the tasks concurrently
        await asyncio.gather(*tasks)

# Run the process in Jupyter
start_time = time.time()
await process_urls()
print("Time taken: ", time.time() - start_time)


Extracted content for https://www.business-standard.com/article/companies/tata-motors-signs-pact-with-ride-hailing-company-uber-to-supply-25-000-evs-123022000923_1.html:
317
Extracted content for https://www.telegraphindia.com/business/tata-motors-to-increase-prices-of-commercial-vehicle/cid/1903713:
271
Extracted content for https://indianexpress.com/article/explained/tata-motors-new-york-stock-exchange-indian-markets-8259994/:
2199
Extracted content for https://economictimes.indiatimes.com/industry/renewables/tata-motors-back-on-funding-street-seeks-around-600m-for-electric-vehiles-business/articleshow/97155362.cms?from=mdr:
5951
Extracted content for https://economictimes.indiatimes.com/news/company/corporate-trends/tata-motors-vp-rajan-amba-appointed-as-md-of-jaguar-land-rover-india/articleshow/98015757.cms?from=mdr:
3396
Extracted content for https://economictimes.indiatimes.com/industry/auto/auto-news/tata-motors-target-of-achieving-net-zero-automotive-debt-by-fy24-on-track-co-gr

11/22/2024 03:45:49 AM - parsed tree length: 0, wrong data type or not valid HTML
11/22/2024 03:45:49 AM - empty HTML tree: None
11/22/2024 03:45:49 AM - discarding data: None


Extracted content for https://economictimes.indiatimes.com/industry/renewables/tata-motors-lines-up-suitors-for-stake-in-ev-business/articleshow/98164751.cms?from=mdr:
2765
Extracted content for https://economictimes.indiatimes.com/industry/renewables/cummins-inc-and-tata-motors-collaborate-to-offer-solutions-in-hydrogen-powered-commercial-vehicle-space/articleshow/95510364.cms?from=mdr:
4506
Extracted content for https://www.businesstoday.in/markets/company-stock/story/tata-motors-loss-narrows-to-rs-945-crore-in-q2-revenue-up-30-yoy-352305-2022-11-09:
2427
Error URL: https://www.mobilityoutlook.com/features/for-tata-motors-disruption-is-the-mantra-in-passenger-vehicles/, Error: Trafilatura failed to extract content
Extracted content for https://www.thehindubusinessline.com/companies/tata-motors-to-drive-in-a-host-of-alternative-fuel-trucks-and-buses-towards-2045-net-zero-goals/article66365628.ece:
2670
Extracted content for https://www.thehindubusinessline.com/companies/tata-motors-ge

11/22/2024 03:45:50 AM - parsed tree length: 1, wrong data type or not valid HTML
11/22/2024 03:45:50 AM - empty HTML tree: None
11/22/2024 03:45:50 AM - discarding data: None


Extracted content for https://www.ndtvprofit.com/markets/tata-motors-bets-big-on-ev-disruption-strategy-to-drive-growth:
6367
Extracted content for https://www.fortuneindia.com/enterprise/ratan-tata-the-man-who-made-tata-motors-a-carmaker-of-consequence-turns-85/110929:
4979
Extracted content for https://www.indiatoday.in/auto/cars/story/tata-motors-sells-over-500000-pvs-in-2022-beats-hyundai-in-december-sales-2316261-2023-01-02:
3110
Error URL: https://www.tatamotors.com/press-releases/tata-motors-delivers-218-winger-veterinary-vans-to-the-government-of-west-bengal/, Error: Trafilatura failed to extract content
Extracted content for https://www.business-standard.com/article/companies/tata-motors-subsidiary-completes-acquisition-of-ford-india-s-sanand-plant-122123001062_1.html:
316


...

Extracted content for https://www.livemint.com/market/stock-market-news/stocks-to-watch-tata-motors-vedanta-ril-federal-bank-iifl-finance-dlf-11710381451564.html:
6107
Extracted content for https://www.livemint.com/companies/news/tata-motors-q4-update-jaguar-land-rovers-retail-sales-surge-81-strongest-performance-since-2009-11712749559712.html:
2142
Extracted content for https://www.livemint.com/auto-news/tata-motors-festive-season-suv-passenger-vehicles-safari-nexon-imd-rainfall-cash-offers-rural-sales-manufacturers-11725890801177.html:
2132
Error URL: https://www.tatamotors.com/press-releases/magenta-mobility-deepens-collaboration-with-tata-motors/, Error: Trafilatura failed to extract content
Extracted content for https://www.livemint.com/market/stock-market-news/tata-motors-vs-mahindra-mahindra-which-auto-major-should-you-pick-for-the-long-term-stock-market-investments-11711045132828.html:
8132
Extracted content for https://www.livemint.com/market/stock-market-news/tata-motors-to-d

11/22/2024 03:46:07 AM - parsed tree length: 1, wrong data type or not valid HTML
11/22/2024 03:46:07 AM - empty HTML tree: None
11/22/2024 03:46:07 AM - discarding data: None
11/22/2024 03:46:07 AM - parsed tree length: 1, wrong data type or not valid HTML
11/22/2024 03:46:07 AM - empty HTML tree: None
11/22/2024 03:46:07 AM - discarding data: None
11/22/2024 03:46:07 AM - parsed tree length: 1, wrong data type or not valid HTML
11/22/2024 03:46:07 AM - empty HTML tree: None
11/22/2024 03:46:07 AM - discarding data: None
11/22/2024 03:46:07 AM - parsed tree length: 1, wrong data type or not valid HTML
11/22/2024 03:46:07 AM - empty HTML tree: None
11/22/2024 03:46:07 AM - discarding data: None


Error URL: https://www.tatamotors.com/press-releases/tata-motors-launches-vidyadhan-and-utkarsha-programs-to-facilitate-higher-education-of-technicians-children/, Error: Trafilatura failed to extract content
Extracted content for https://www.tatamotors.com/press-releases/tata-motors-launches-its-mid-suv-curvv-at-a-starting-price-of-rs-9-99-lakh/:
7046
Extracted content for https://www.tatamotors.com/press-releases/tata-motors-registered-total-sales-of-215034-units-in-q2-fy25/:
4702
Error URL: https://www.tatamotors.com/press-releases/tata-motors-announces-price-increase-of-its-commercial-vehicles-from-july-2024/, Error: Trafilatura failed to extract content
Error URL: https://www.tatamotors.com/press-releases/tata-motors-group-global-wholesales-at-329847-in-q1-fy25/, Error: Trafilatura failed to extract content
Extracted content for https://www.tatamotors.com/press-releases/tata-motors-registered-total-sales-of-71693-units-in-august-2024/:
1673
Error URL: https://www.tatamotors.com/pre

11/22/2024 03:46:08 AM - parsed tree length: 1, wrong data type or not valid HTML
11/22/2024 03:46:08 AM - empty HTML tree: None
11/22/2024 03:46:08 AM - discarding data: None


Extracted content for https://www.tatamotors.com/press-releases/tata-motors-launches-festival-of-cars-with-incredible-prices-for-its-cars-suvs/:
2025
Error URL: https://www.tatamotors.com/press-releases/tata-motors-celebrates-900000th-vehicle-rollout-from-its-lucknow-facility/, Error: Trafilatura failed to extract content
Extracted content for https://www.tatamotors.com/press-releases/20-lakh-suvs-strong-tata-motors-indias-leading-suv-manufacturer-celebrates-a-historic-milestone-of-its-suv-legacy/:
2830
Extracted content for https://www.tatamotors.com/press-releases/tata-motors-signs-mou-with-esaf-small-finance-bank-for-commercial-vehicle-financing/:
2182
Extracted content for https://www.tatamotors.com/press-releases/tata-power-renewable-energy-and-tata-motors-sign-a-mou-to-set-up-200-fast-charging-stations/:
3462
Extracted content for https://www.tatamotors.com/press-releases/tata-motors-registered-total-sales-of-77521-units-in-april-2024/:
1652
Extracted content for https://www.tata

In [1]:
# Gemini

In [1]:
pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!where python

e:\Intern\Minerva\Final_Dashboard_GIT\venv\Scripts\python.exe
C:\Users\Sahal\AppData\Local\Programs\Python\Python312\python.exe
C:\Users\Sahal\AppData\Local\Microsoft\WindowsApps\python.exe


In [1]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Write a story about a magic backpack. in 5 lines")
print(response.text)

e:\Intern\Minerva\Final_Dashboard_GIT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Elara found a worn leather backpack in the attic; it hummed faintly.  Inside, she discovered shimmering silks, transforming into anything she imagined.  A castle appeared for tea parties, a dragon for imaginary battles.  But one day, a mischievous imp emerged, demanding a wish in exchange for its silence.  Elara, clever and brave, outwitted the imp, keeping her magical backpack and its secrets.



In [1]:
a= [3,4]
print(a*3)

[3, 4, 3, 4, 3, 4]


# Playwright Test

In [3]:
import pandas as pd

df = pd.read_excel(r"E:\Intern\Minerva\LLM API\TATA Motors.xlsx")
links = list(df["Link"][:500])
link = links[0]

In [7]:
from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch()
    page = browser.new_page()
    page.goto("http://playwright.dev")
    print(page.title())
    browser.close()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

# Report Generator Gemini

In [9]:
import pandas as pd
import json

# Load the Excel file into a DataFrame
df = pd.read_excel(r"E:\Intern\Minerva\Final_Dashboard_GIT\QnA_debugging_output.xlsx")

# Initialize an empty dictionary to store the structured data
structured_data = {}
current_question = None
question_counter = 1  # Counter for numbering the questions

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    if pd.notna(row['Question']):  # Check if the row has a new question
        # Create a key like "Q1", "Q2", ..., for each question
        question_key = f"Q{question_counter}"
        structured_data[question_key] = {
            "question": row['Question'],  # Store the full question
            "rows": []  # Initialize an empty list for rows
        }
        current_question = question_key  # Update the current question key
        question_counter += 1  # Increment the counter
    else:
        # Append the row data to the current question's list of rows
        if current_question is not None:
            structured_data[current_question]["rows"].append({
                "Date": row['Date'],
                "Link": row['Link'],
                "Answer": row['Answer']
            })

# Convert the structured dictionary to JSON format
QnA_input = json.dumps(structured_data, indent=4)

# Print or save the JSON data
print(QnA_input)


{
    "Q1": {
        "question": "Are there any regulatory or legal issues faced by the Company or its subsidiaries?",
        "rows": [
            {
                "Date": "2023-01-13",
                "Link": "https://www.india.com/entertainment/tv-news-sheezan-khan-gets-replaced-by-abhishek-nigam-in-ali-baba-dastaan-e-kabul-tanisha-sharmas-character-not-finalised-5851949/",
                "Answer": "No"
            },
            {
                "Date": "2023-06-26",
                "Link": "https://www.zeebiz.com/market-news/news-mastermind-of-estimated-rs-1500-crore-sms-stock-tip-scam-hanif-shekh-likely-absconding-from-india-241823",
                "Answer": "Yes\nGujarat Toolroom is not mentioned in this news article. This news article discusses a large-scale stock manipulation scheme involving multiple companies and individuals, including alleged involvement of promoters of some small companies. SEBI is investigating Hanif Shekh, the alleged mastermind of the scheme, and 

In [23]:
def generate_system_instruction(company_name):
    return f"""
### **Prompt**:

**Company_name:** {company_name}  

You are an expert financial analyst tasked with generating a concise and standardized financial analysis report for the given company based on the extracted data. The analysis should focus on {company_name}'s financial health, industry performance, and key operational aspects. The provided data will include questions, answers, references, and dates.

#### **Key Questions to Address**:

1. **Are there any regulatory or legal issues faced by {company_name} or its subsidiaries?**  
2. **Are there any legal issues involving the promoters of {company_name}?**  
3. **Has {company_name} faced significant employee attrition or changes in key management in the past 2 years?**  
4. **Is the industry in which {company_name} operates currently experiencing a slowdown?**  
5. **Is {company_name} overvalued compared to its peers?**  
6. **Are there any significant upcoming events, partnerships, or product launches that could impact {company_name}'s performance?**  
7. **Has {company_name} shown consistent growth in revenue, operating profit margins, and net profit margins year-on-year for the past 2 years, and how does this compare to the industry growth rate?**  
8. **Has {company_name}'s debt increased or decreased over the past 2 years?**  
9. **Has {company_name}'s capacity utilization improved or declined in the past 2 years, and has the company added any capacity in the last 3 years?**  
10. **Has the promoter stake in {company_name} increased or decreased over the years?**  
11. **Has the institutional stake in {company_name} increased or decreased over the years?**  
12. **How many analysts are tracking {company_name} stock, and what is the percentage upside on their target price?**  

---

### **Output Structure**:

#### **Title**:
- Largest font size, e.g., *"{company_name} Financial Analysis Report"*

*This report provides an in-depth financial analysis of {company_name}, focusing on key performance indicators, industry trends, and potential factors influencing future growth. The findings are based on the latest publicly available data and address critical areas such as financial health, market valuation, and stakeholder dynamics.*

#### **Sections**:  
Each question will have a dedicated section using the following format:

1. **Heading**:  
   Use a concise title summarizing the question (e.g., *Legal Issues*, *Key Management Changes*, etc.).

2. **Summary**:  
   - Provide a maximum 5-sentence summary based on the available findings.  
   - If data is unavailable, explicitly state: *"No relevant information available."*

3. **Key Points**:  
   - Highlight the findings in a clear, bullet-pointed format (max 3–4 points) with references.  
   - Include actionable insights and context with proper attribution URLs using this markdown format:  
     [Source Name - Date](url)  
     e.g.,  
     - HDFC Bank's stock hit an all-time high above ₹1,800 on November 25, 2024, driven by a surge in trading volumes due to the MSCI November rebalancing ([Moneycontrol - 2024-04-01](https://www.moneycontrol.com/news/business/markets/hdfc-bank-stock-hits-all-time-high-above-rs-1800-as-trading-volumes-surge-on-msci-rejig-inflows-12875473.html)).

---

### **Example Output**:

```markdown
# {company_name} Financial Analysis Report

*This report provides an in-depth financial analysis of {company_name}, focusing on key performance indicators, industry trends, and potential factors influencing future growth. The findings are based on structured data addressing critical areas such as financial health, market valuation, and stakeholder dynamics.*

---

## **1. Legal and Regulatory Issues**  
**Summary**:  
{company_name} is facing legal challenges, including regulatory scrutiny. There are no major cases filed against its subsidiaries at the moment.

**Key Points**:  
- {company_name} facing regulatory scrutiny in ongoing investigations ([Source - 2024-06-01](https://example.com/link1))  
- No major cases filed against subsidiaries ([Source - 2024-06-05](https://example.com/link2))  

---

## **2. Promoter Legal Concerns**  
**Summary**:  
Promoter of the company is under investigation for tax-related issues.

**Key Points**:  
- Promoter X under investigation for tax evasion ([Source - 2024-06-03](https://example.com/link2))  

---

## **3. Key Management Changes**  
**Summary**:  
No relevant information available.

---

## **4. Industry Slowdown**  
**Summary**:  
No relevant information available.

---

## **5. Company Valuation**  
**Summary**:  
No relevant information available.

---

## **6. Upcoming Events and Product Launches**  
**Summary**:  
No relevant information available.

---

## **7. Revenue and Profit Margins**  
**Summary**:  
{company_name} has demonstrated exceptional year-on-year growth in revenue and profit margins over the past year, often exceeding several thousand percent. While precise industry growth rates are unavailable, the company's growth significantly surpasses the average.

**Key Points**:  
- Massive year-on-year growth in revenue and net profits reported across multiple sources. ([Source - 2024-12-09](https://example.com/link3))  
- Industry growth is far behind the company's performance ([Business Times - 2024-12-10](https://example.com/link4))  

---

## **8. Debt Trends**  
**Summary**:  
No relevant information available.

---

## **9. Capacity Utilization**  
**Summary**:  
No relevant information available.

---

## **10. Promoter Stake**  
**Summary**:  
No relevant information available.

---

## **11. Institutional Stake**  
**Summary**:  
No relevant information available.

---

## **12. Analyst Coverage**  
**Summary**:  
No relevant information available.

---

**Disclaimer:** This report relies on publicly available information and is not intended as investment advice. Readers should conduct independent due diligence before making investment decisions.

---

### **Additional Guidelines**:

1. **Clarity and Informative**:  
   - Present actionable insights without being too long or too short and avoid redundant details.  
   - Emphasize findings that are most impactful for understanding {company_name}'s performance and prospects.

2. **Formatting and Uniformity**:  
   - If there are mutliple sources that mention the same point, include the reference for a maximum of 3 sources:
      - eg. - Massive year-on-year growth in revenue and net profits reported across multiple sources. ([Source 1](link1)), ([Source 2](link2)), ([Source 3](link3)).
   - Maintain consistent formatting across all sections for clarity.  
   
3. **Relevance**:  
   - Exclude extraneous details that do not directly address the questions.  
   - Focus on insights that provide significant value for stakeholders.
"""



In [24]:
import os
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold


genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
    }

company_name = "Gujarat Toolroom"

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  safety_settings=safety_settings,
  system_instruction= generate_system_instruction(company_name))

chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message(json_data)

print(response.text)

# Gujarat Toolroom Financial Analysis Report

*This report provides an in-depth financial analysis of Gujarat Toolroom, focusing on key performance indicators, industry trends, and potential factors influencing future growth. The findings are based on the latest publicly available data and address critical areas such as financial health, market valuation, and stakeholder dynamics.*

---

## 1. Legal and Regulatory Issues

**Summary**:
While one news article mentions a large-scale stock manipulation scheme under SEBI investigation, there's no direct implication or mention of Gujarat Toolroom's involvement.  Most sources report no regulatory or legal issues impacting Gujarat Toolroom or its subsidiaries.  One source indicates potential management concerns.

**Key Points**:
- A SEBI investigation into a stock manipulation scheme mentions unnamed small companies potentially involved ([ZeeBiz - 2023-06-26](https://www.zeebiz.com/market-news/news-mastermind-of-estimated-rs-1500-crore-sms-sto

# Gujarat Toolroom Financial Analysis Report

*This report provides an in-depth financial analysis of Gujarat Toolroom, focusing on key performance indicators, industry trends, and potential factors influencing future growth. The findings are based on the latest publicly available data and address critical areas such as financial health, market valuation, and stakeholder dynamics.*

---

## 1. Legal and Regulatory Issues

**Summary**:
While one news article mentions a large-scale stock manipulation scheme under SEBI investigation, there's no direct implication or mention of Gujarat Toolroom's involvement.  Most sources report no regulatory or legal issues impacting Gujarat Toolroom or its subsidiaries.  One source indicates potential management concerns.

**Key Points**:
- A SEBI investigation into a stock manipulation scheme mentions unnamed small companies potentially involved ([ZeeBiz - 2023-06-26](https://www.zeebiz.com/market-news/news-mastermind-of-estimated-rs-1500-crore-sms-stock-tip-scam-hanif-shekh-likely-absconding-from-india-241823)).
- No direct involvement of Gujarat Toolroom in the above-mentioned investigation is reported across numerous other sources.
- MarketsMojo downgraded Gujarat Toolroom to a "Hold" rating due to management concerns and high valuation ([MarketsMojo - 2024-03-14](https://www.marketsmojo.com/news/stock-recommendation/gujarat-toolroom-downgraded-to-hold-by-marketsmojo-due-to-management-concerns-and-high-valuation-100358)).


---

## 2. Promoter Legal Concerns

**Summary**:
One source mentions that promoters of some small companies are under investigation for involvement in a stock manipulation scheme.  However, no specific legal issues are directly linked to Gujarat Toolroom's promoters in the majority of reviewed sources.

**Key Points**:
-  Promoters of several unnamed companies are implicated in a SEBI investigation related to a stock tip scam ([ZeeBiz - 2023-06-26](https://www.zeebiz.com/market-news/news-mastermind-of-estimated-rs-1500-crore-sms-stock-tip-scam-hanif-shekh-likely-absconding-from-india-241823)).
- No other source provides information directly implicating Gujarat Toolroom's promoters in legal issues.


---

## 3. Key Management Changes

**Summary**:
Significant changes in key management occurred within the past two years.  Anicattu Issac George, P V Rama Seshu, and Kalpesh Anilbhai Malvi all resigned from their positions in 2023.  This may impact the company's operational stability and future performance.  Later reports show that Rakesh Sharma was approved as Managing Director.

**Key Points**:
- Anicattu Issac George and P V Rama Seshu resigned in August 2023 ([Equitymaster - 2023-10-02](https://www.equitymaster.com/detail.asp?date=10/02/2023&story=3&title=Top-Performing-Penny-Stocks-of-September-2023)).
- Kalpesh Anilbhai Malvi (MD and CFO) resigned in November 2023 ([The Hindu BusinessLine - 2023-11-13](https://www.thehindubusinessline.com/markets/share-market-highlights-13-november-2023/article67527403.ece)).
- Rakesh Sharma's appointment as Managing Director was approved ([Marketscreener - 2024-08-16](https://www.marketscreener.com/quote/stock/GUJARAT-TOOLROOM-LIMITED-46729180/news/Gujarat-Toolroom-Limited-Approves-Regularization-of-Rakesh-Sharma-as-Managing-Director-47670737/)).


---

## 4. Industry Slowdown

**Summary**:
No relevant information available on an industry-wide slowdown affecting Gujarat Toolroom's sector.  The company's performance indicates growth, contradicting a potential slowdown.

---

## 5. Company Valuation

**Summary**:
Analyst opinions on Gujarat Toolroom's valuation are mixed. Some sources suggest overvaluation based on high Price-to-Book ratios and PEG ratios, while others indicate undervaluation relative to peers based on the P/E ratio and financial performance.  Further independent analysis is needed.

**Key Points**:
- MarketsMojo cited overvaluation in March 2024 due to high Price-to-Book and PEG ratios ([MarketsMojo - 2024-03-14](https://www.marketsmojo.com/news/stock-recommendation/gujarat-toolroom-downgraded-to-hold-by-marketsmojo-due-to-management-concerns-and-high-valuation-100358)).
- Times Now News indicated undervaluation in May 2024 based on the P/E ratio ([Times Now News - 2024-05-07](https://www.timesnownews.com/business-economy/companies/gujarat-toolroom-q4-results-2024-company-reports-massive-revenue-surge-in-fy24-details-article-109923860)).
- Qrius and Livemint reported significant undervaluation compared to industry peers in December 2024 based on PE ratios ([Qrius - 2024-12-09](https://qrius.com/foreign-institutions-bet-on-these-three-penny-stocks-amid-their-massive-selloff/), [Livemint - 2024-12-09](https://www.livemint.com/market/stock-market-news/penny-stocks-fii-investments-fii-selloff-gujarat-toolroom-ltd-spicejet-ltd-sera-investments-and-finance-india-ltd-11733725968399.html)).


---

## 6. Upcoming Events and Product Launches

**Summary**:
Several significant upcoming events are projected to impact Gujarat Toolroom's performance. These include expansion into mining (Zambia), renewable energy (hybrid power plant), and international markets (Dubai subsidiary).  The company also secured substantial orders from Reliance Industries and other international clients.

**Key Points**:
-  Acquisition of mines in Zambia ([ZeeBiz - 2024-01-02](https://www.zeebiz.com/companies/news-gujarat-toolroom-acquires-mines-in-zambia-270696)).
- Development of a large-scale hybrid renewable energy power plant ([Economic Times - 2024-05-02](https://economictimes.indiatimes.com/industry/renewables/gujarat-toolroom-acquires-65-acre-land-for-rs-570-cr-hybrid-energy-project/articleshow/109792508.cms?from=mdr)).
- Significant orders secured from Reliance Industries ([India Today - 2024-04-23](https://www.indiatoday.in/information/story/small-cap-company-inks-rs2-billion-deal-with-reliance-industries-stocks-rally-2530595-2024-04-23)).
-  Expansion into international markets (Dubai subsidiary) ([Times Now News - 2023-09-25](https://www.timesnownews.com/business-economy/companies/gujarat-toolroom-limited-reveals-investment-plans-in-dubai-details-article-103925711)).


---

## 7. Revenue and Profit Margins

**Summary**:
Gujarat Toolroom exhibited exceptional year-on-year growth in revenue and profit margins over the past two years, often exceeding several thousand percent. Precise industry growth rates are unavailable from the provided sources, however,  the company's growth significantly surpasses reported averages.

**Key Points**:
- Massive year-on-year growth in revenue and net profits reported ([India.com - 2024-01-02](https://www.india.com/business/stock-market-news-gujarat-toolroom-limited-shares-gets-buy-rating-all-you-need-to-know-6627834/), [Livemint - 2024-01-04](https://www.livemint.com/market/stock-market-news/penny-stock-gujarat-toolroom-share-price-skyrockets-gives-multibagger-return-in-just-2-years-11704271537694.html)).
-  Growth significantly outpaces available data on industry growth rates.


---

## 8. Debt Trends

**Summary**:
Gujarat Toolroom's debt has significantly decreased over the past three years, with reports indicating it is now nearly debt-free.

**Key Points**:
-  Debt significantly reduced, company almost debt-free ([DNPindia - 2023-11-13](https://www.dnpindia.in/current-affairs/business/gujarat-toolroom-stock-soars-over-1500-percentage-in-just-one-year-with-striking-q2-performance-what-are-you-waiting-for/330071/)).
- Low Debt-to-Equity ratio (0.07 times) indicates a very low debt burden ([MarketsMojo - 2024-11-05](https://www.marketsmojo.com/news/stock-recommendation/gujarat-toolroom-downgraded-to-hold-by-marketsmojo-despite-strong-financial-performance-296740)).


---

## 9. Capacity Utilization

**Summary**:
No relevant information available regarding Gujarat Toolroom's capacity utilization or capacity additions over the past three years.

---

## 10. Promoter Stake

**Summary**:
Information on promoter stake changes is inconsistent across sources. Some articles indicate a decrease, while others suggest an increase or remain silent.  More reliable data is needed for a conclusive statement.

**Key Points**:
-  Decreased by -0.51% in one previous quarter ([MarketsMojo - 2024-02-19](https://www.marketsmojo.com/news/stock-recommendation/gujarat-toolroom-receives-hold-rating-upgrade-46718)).
-  Increased from 70% to 56% in one period ([Qrius - 2024-12-09](https://qrius.com/foreign-institutions-bet-on-these-three-penny-stocks-amid-their-massive-selloff/)).
- Decreased from 70% to 56% in another account ([Livemint - 2024-12-09](https://www.livemint.com/market/stock-market-news/penny-stocks-fii-investments-fii-selloff-gujarat-toolroom-ltd-spicejet-ltd-sera-investments-and-finance-india-ltd-11733725968399.html)).


---

## 11. Institutional Stake

**Summary**:
The institutional stake in Gujarat Toolroom has increased significantly, largely due to a successful Qualified Institutional Placement (QIP). FIIs notably increased their holdings.

**Key Points**:
-  Increased due to a QIP ([Indiahood - 2024-10-11](https://indiahood.com/gujarat-toolroom-raises-funds-through-qip-stock-hits-5-upper-circuit/)).
- FIIs increased their stake from 0% to 27.15% ([Tradebrains - 2024-10-22](https://tradebrains.in/microcap-stocks-in-which-fiis-have-increased-their-stake-to-keep-on-your-radar/)).


---

## 12. Analyst Coverage

**Summary**:
While some analysts have issued buy ratings and provided target prices, suggesting potential upside,  a consistent number of tracking analysts or a definitive percentage upside is unavailable from the provided sources.  Opinions vary significantly.

**Key Points**:
-  Target price of Rs 70+ suggested, indicating potential upside ([India.com - 2024-01-02](https://www.india.com/business/stock-market-news-gujarat-toolroom-limited-shares-gets-buy-rating-all-you-need-to-know-6627834/)).
- Long-term target price of Rs 90 set by ProfitMart Securities ([India TV News - 2024-04-10](https://www.indiatvnews.com/business/news/multibagger-smallcap-stock-declares-100-per-cent-dividend-payout-sets-record-date-dividend-bse-filing-2024-04-10-925528)).
-  MarketsMojo issued a "Hold" rating in multiple instances, suggesting no significant upside based on their analysis ([MarketsMojo - 2024-03-17](https://www.marketsmojo.com/news/stocks-in-action/gujarat-toolrooms-stock-price-hits-52-week-high-outperforms-sector-by-4-60409), [MarketsMojo - 2024-08-16](https://www.marketsmojo.com/news/result-analysis/gujarat-toolrooms-q1-financial-results-show-impressive-growth-and-positive-outlook-261882)).


---

**Disclaimer:** This report relies on publicly available information and is not intended as investment advice. Readers should conduct independent due diligence before making investment decisions.

# QnA extracted df to json

In [2]:
import json
import pandas as pd

def convert_df_to_json(df: pd.DataFrame) -> str:
    # Initialize an empty dictionary to store the structured data
    structured_data = {}
    current_question = None
    question_counter = 1  # Counter for numbering the questions

    # Iterate through each row of the DataFrame
    for index, row in df.iterrows():
        if pd.notna(row['Question']):  # Check if the row has a new question
            # Create a key like "Q1", "Q2", ..., for each question
            question_key = f"Q{question_counter}"
            structured_data[question_key] = {
                "question": row['Question'],  # Store the full question
                "rows": []  # Initialize an empty list for rows
            }
            current_question = question_key  # Update the current question key
            question_counter += 1  # Increment the counter
        else:
            # Append the row data to the current question's list of rows
            if current_question is not None:
                structured_data[current_question]["rows"].append({
                    "Date": row['Date'],
                    "Link": row['Link'],
                    "Answer": row['Answer']
                })

    # Convert the structured dictionary to a JSON string with indentation
    json_result = json.dumps(structured_data)

    return json_result  # Returning the JSON string

# Example usage
df = pd.read_excel(r"E:\Intern\Minerva\Final_Dashboard_GIT\QnA_debugging_output.xlsx")
json_output = convert_df_to_json(df)
json_output

'{"Q1": {"question": "Are there any regulatory or legal issues faced by the Company or its subsidiaries?", "rows": [{"Date": "2024-07-24", "Link": "https://tradebrains.in/features/penny-stock-hits-5-upper-circuit-after-securing-international-order-worth-150-cr/", "Answer": "No\\nNo regulatory or legal issues are mentioned in the article."}, {"Date": "2024-07-25", "Link": "https://www.india.com/business/gujarat-toolroom-shares-in-focus-as-company-announced-fresh-order-of-rs-1-5-billion-order-7114365/", "Answer": "No"}, {"Date": "2024-08-06", "Link": "https://www.india.com/business/share-market-news-gujarat-toolroom-lands-rs-1-14-billion-impex-order-check-details-here-7145429/", "Answer": "No\\nNo regulatory or legal issues are mentioned in the article."}, {"Date": "2024-08-16", "Link": "https://www.marketscreener.com/quote/stock/GUJARAT-TOOLROOM-LIMITED-46729180/news/Gujarat-Toolroom-Limited-Approves-Regularization-of-Rakesh-Sharma-as-Managing-Director-47670737/", "Answer": "No"}, {"Dat

# Gemini Report gen

In [8]:
import os
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

def generate_financial_report(company_name: str, user_input: str) -> str:
    # Configure the API with your GEMINI API key
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])

    # Set up generation configuration
    generation_config = {
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
    }

    # Set up safety settings
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
    }

    # Generate system instruction for the given company
    system_instruction = generate_system_instruction(company_name)

    # Create the model
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config,
        safety_settings=safety_settings,
        system_instruction=system_instruction
    )

    # Start chat session
    chat_session = model.start_chat(
        history=[]
    )

    # Send the user input message to get the output
    response = chat_session.send_message(user_input)

    # Return the response text
    return response.text

def save_markdown(content: str, filename: str):
    # Open the file in write mode ('w'). If the file exists, it will be overwritten.
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

generate_financial_report("Gujarat Toolroom", json_output)

'```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Gujarat Toolroom Financial Analysis Report</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            margin: 0;\n            padding: 0;\n            background-color: #f4f4f4;\n            color: #333;\n        }\n        .container {\n            width: 80%;\n            margin: 20px auto;\n            background-color: #fff;\n            padding: 20px;\n            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);\n        }\n        h1 {\n            text-align: center;\n            font-size: 2em;\n            color: #0044cc;\n        }\n        h2 {\n            font-size: 1.5em;\n            color: #333;\n        }\n        h3 {\n            font-size: 1.2em;\n            color: #444;\n        }\n        p {\n            line-height: 1.6;\n            margin-bottom: 10px;\n        }\n  